In [ ]:
!pip install -qqq simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 k

In [ ]:
import pandas as pd
from simpletransformers.question_answering import QuestionAnsweringModel
from sklearn.model_selection import train_test_split
import wandb

In [ ]:
data = pd.read_csv("train.csv")
train_df, eval_df = train_test_split(data, test_size=0.25, random_state=42)

In [ ]:
model_configs = {
    "bert": {
        "model_type": "bert",
        "model_name": "bert-base-uncased",
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "wandb_project": "MedQuad_QA_BERT"
    },
    "mobilebert": {
        "model_type": "bert",
        "model_name": "google/mobilebert-uncased",
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "wandb_project": "MedQuad_QA_MobileBERT"
    },
    "roberta": {
        "model_type": "roberta",
        "model_name": "roberta-base",
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "wandb_project": "MedQuad_QA_RoBERTa"
    }
}

In [ ]:
hyperparameters = [
    {"num_train_epochs": 3, "learning_rate": 5e-5, "dropout": 0.3},
    {"num_train_epochs": 3, "learning_rate": 5e-5, "dropout": 0.7}
]

In [ ]:
def preprocess_data(df):
    examples = []
    for idx, row in df.iterrows():
        context = row["Answer"]
        qas = [{"question": row["Question"], "id": idx, "answers": [{"text": row["Answer"], "answer_start": 0}]}]
        examples.append({"context": context, "qas": qas})
    return examples
for model_name, config in model_configs.items():
    for hp in hyperparameters:
        config.update(hp)
        train_examples = preprocess_data(train_df)
        eval_examples = preprocess_data(eval_df)
        wandb.init(project=config["wandb_project"], config=config)
        model = QuestionAnsweringModel(model_type=config["model_type"],
                                       model_name=config["model_name"],
                                       args=config)
        output_dir = f"{model_name} {hp}"
        model.train_model(train_examples, output_dir=output_dir)
        result = model.eval_model(eval_examples)
        wandb.log({"eval_results": result})